# Vision Pipeline - Google Colab 컨트롤러

이 노트북을 사용하면 Google Colab에서 GPU를 사용하여 vision pipeline을 실행할 수 있습니다.
Google Drive를 마운트하여 저장 공간으로 사용하고 자동으로 환경을 설정합니다.

## 1. 환경 설정

In [ ]:
# @title Clone Repository & Install Dependencies (Colab-safe)
import os
import subprocess
import sys

# --- Configuration ---
REPO_URL = "https://github.com/ppopgi-pang/ppopgipang-vision-auto-labeler.git"  # @param {type:"string"}
BRANCH = "main"  # @param {type:"string"}
# ---------------------


def run_cmd(cmd):
    print("+", " ".join(cmd))
    subprocess.run(cmd, check=True)


def ensure_playwright_package():
    try:
        import playwright  # noqa: F401
        return
    except Exception:
        print("Installing Playwright Python package...")
        run_cmd([sys.executable, "-m", "pip", "install", "playwright"])


def install_playwright_browsers():
    print("Installing Playwright and system dependencies...")
    try:
        run_cmd([sys.executable, "-m", "playwright", "install", "--with-deps", "chromium"])
    except subprocess.CalledProcessError:
        print("Playwright system dependency install failed; retrying browser-only install.")
        run_cmd([sys.executable, "-m", "playwright", "install", "chromium"])


# 1. Ensure Playwright is available and install browsers (FIRST)
ensure_playwright_package()
install_playwright_browsers()

# 2. Clone repository into a stable base directory
repo_name = REPO_URL.split("/")[-1].replace(".git", "")
BASE_DIR = "/content"
if not os.path.exists(BASE_DIR):
    BASE_DIR = os.getcwd()
repo_path = os.path.join(BASE_DIR, repo_name)

if not os.path.exists(repo_path):
    print(f"Cloning {REPO_URL} into {repo_path}...")
    run_cmd(["git", "clone", "-b", BRANCH, REPO_URL, repo_path])
else:
    print(f"Repository already exists at {repo_path}.")

# 3. Move into the project directory
os.chdir(repo_path)

# 4. Install Python dependencies
print("Installing Python dependencies...")
if os.path.exists("requirements.txt"):
    run_cmd([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
else:
    print("requirements.txt not found, installing fallback dependencies...")
    fallback_packages = [
        "pyyaml",
        "pydantic",
        "pydantic-settings",
        "python-dotenv",
        "requests",
        "pillow",
        "numpy",
        "opencv-python-headless",
        "ImageHash",
        "torch",
        "torchvision",
        "transformers",
        "ultralytics",
        "openai",
        "playwright",
        "nest_asyncio",
    ]
    run_cmd([sys.executable, "-m", "pip", "install", *fallback_packages])

print("Setup completed successfully.")


## 1.1 API 키 설정
**중요:** 프로젝트 모듈을 import하기 전에 이 셀을 실행하여 키가 올바르게 로드되도록 하세요.

In [ ]:
import os

# @markdown ### API 키
# @markdown API 키를 여기에 입력하세요. 크롤링 및 LLM 검증에 필요합니다.
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# 환경 변수에 주입
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 유효성 검증 경고
if "YOUR_" in OPENAI_API_KEY:
    print("WARNING: OPENAI_API_KEY에 기본 플레이스홀더가 감지되었습니다. 실제 키로 업데이트하세요.")
    
print("환경 변수 설정 완료.")

## 2. GPU 확인

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"SUCCESS: GPU 사용 가능 - {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: GPU를 찾을 수 없습니다. Runtime > Change runtime type > Hardware accelerator > GPU에서 활성화하세요.")

## 3. Google Drive 마운트 및 경로 설정

In [ ]:
from google.colab import drive
import sys
import os

# Drive 마운트 (종료 후 결과 복사용)
drive.mount('/content/drive')

# 경로 설정
PROJECT_ROOT = os.getcwd()
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added {PROJECT_ROOT} to sys.path")

# 중요: vision_pipeline을 sys.path에 추가하여 'pipelines', 'domain' 등에서 import 가능하도록 함
PIPELINE_ROOT = os.path.join(PROJECT_ROOT, 'vision_pipeline')
if PIPELINE_ROOT not in sys.path:
    sys.path.append(PIPELINE_ROOT)
    print(f"Added {PIPELINE_ROOT} to sys.path")

# 로컬 출력 디렉토리 (실행 중)
LOCAL_OUTPUT_DIR = "/content/vision_pipeline_data"  # @param {type:"string"}
os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)

# Drive로 이동할 최종 출력 디렉토리
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/vision_pipeline_data"  # @param {type:"string"}

# config.py 기본값을 오버라이드하기 위한 환경 변수 설정
os.environ["OUTPUT_DIR"] = LOCAL_OUTPUT_DIR
print(f"로컬 출력 디렉토리 설정: {LOCAL_OUTPUT_DIR}")
print(f"Drive 출력 디렉토리(종료 후 복사): {DRIVE_OUTPUT_DIR}")


## 4. 파이프라인 실행

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import os
import shutil
import tarfile
import time

from vision_pipeline.domain.job import Job
from vision_pipeline.services.pipeline_runner import PipelineRunner
from vision_pipeline.config import settings
import logging

LOCAL_OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/content/vision_pipeline_data")
DRIVE_OUTPUT_DIR = globals().get("DRIVE_OUTPUT_DIR", "/content/drive/MyDrive/vision_pipeline_data")

# --- 실행 설정 ---
KEYWORDS = """
Disney Mickey Mouse plush, Disney Minnie Mouse plush, Disney Donald Duck plush, Disney Daisy Duck plush, Disney Goofy plush, Disney Pluto plush, Disney Chip plush, Disney Dale plush, Disney Elsa plush, Disney Anna plush, Disney Olaf plush, Disney Rapunzel plush, Disney Moana plush, Disney Stitch plush, Disney Pooh plush, Disney Winnie the Pooh plush, Disney Piglet plush, Disney Tigger plush, Disney Eeyore plush, Disney Simba plush, Disney Nala plush, Disney Pumbaa plush, Disney Timon plush, Disney Buzz plush, Disney Woody plush, Disney Jessie plush, Disney Rex plush, Disney Forky plush, Disney Hamm plush, Disney Alien plush, Disney Mike plush, Disney Sulley plush, Disney Wall-E plush, Disney Eve plush, Disney Nemo plush, Disney Dory plush, Disney Remy plush, Disney Baymax plush, Disney Wreck-It Ralph plush, Disney Vanellope plush, Disney Judy Hopps plush, Disney Nick Wilde plush, Disney Aladdin plush, Disney Jasmine plush, Disney Genie plush, Disney Cinderella plush, Disney Snow White plush, Disney Aurora plush, Disney Ariel plush, Disney Flounder plush, Disney Sebastian plush, Disney Belle plush, Disney Beast plush, Disney Mulan plush, Disney Mushu plush, Disney Tiana plush, Disney Merida plush, Disney Dumbo plush, Disney Bambi plush, Disney Pinocchio plush, Disney Alice plush, Disney Cheshire Cat plush, Disney Mirabel plush, Disney Bruno plush, Disney Luisa plush, Disney Isabela plush, Disney Luca plush, Disney Alberto plush, Disney Asha plush, Sanrio Hello Kitty plush, Sanrio My Melody plush, Sanrio Kuromi plush, Sanrio Cinnamoroll plush, Sanrio Pompompurin plush, Sanrio Pochacco plush, Sanrio Keroppi plush, Sanrio Badtz-Maru plush, Sanrio Hangyodon plush, Sanrio Little Twin Stars plush, Sanrio Kiki plush, Sanrio Lala plush, Sanrio Marron Cream plush, Sanrio Usahana plush, Sanrio Gudetama plush, Sanrio My Sweet Piano plush, Ghibli Totoro plush, Ghibli Medium Totoro plush, Ghibli Small Totoro plush, Ghibli Catbus plush, Ghibli Jiji plush, Ghibli Jiji Black Cat plush, Ghibli No-Face plush, Ghibli Haku plush, Ghibli Chihiro plush, Ghibli Yubaba plush, Ghibli Ponyo plush, Ghibli Sosuke plush, Ghibli Howl plush, Ghibli Sophie plush, Ghibli Calcifer plush, Ghibli San plush, Ghibli Ashitaka plush, Ghibli Yakul plush, Ghibli Kodama plush, Ghibli Nausicaa plush, Ghibli Teto plush, Ghibli Mei plush, Ghibli Satsuki plush, Ghibli Arrietty plush, Ghibli Baron plush, Ghibli Muta plush, Pokemon Pikachu plush, Pokemon Eevee plush, Pokemon Charmander plush, Pokemon Charizard plush, Pokemon Squirtle plush, Pokemon Blastoise plush, Pokemon Bulbasaur plush, Pokemon Snorlax plush, Pokemon Mew plush, Pokemon Mewtwo plush, Pokemon Dragonite plush, Pokemon Jigglypuff plush, Pokemon Pichu plush, Pokemon Lucario plush, Pokemon Lugia plush, Pokemon Ho-Oh plush, Pokemon Growlithe plush, Pokemon Arcanine plush, Pokemon Magikarp plush, Pokemon Gyarados plush, Pokemon Emolga plush, Pokemon Psyduck plush, Pokemon Sylveon plush, Pokemon Umbreon plush, Pokemon Glaceon plush, Pokemon Leafeon plush, Pokemon Flareon plush, Pokemon Vaporeon plush, Pokemon Jolteon plush, Pokemon Ditto plush, Pokemon Raichu plush, Crayon Shin-chan Shin-chan plush, Crayon Shin-chan Shiro plush, Crayon Shin-chan Kazama plush, Crayon Shin-chan Nene plush, Crayon Shin-chan Masao plush, Crayon Shin-chan Bo plush, Crayon Shin-chan Himawari plush, Crayon Shin-chan Misae plush, Crayon Shin-chan Hiroshi plush, Crayon Shin-chan Ai plush, Crayon Shin-chan Action Mask plush, Bonobono Bonobono plush, Bonobono Raccoon plush, Bonobono Porori plush, Bonobono Chipmunk plush, Bonobono Araiguma plush, Chiikawa Chiikawa plush, Chiikawa Hachiware plush, Chiikawa Usagi plush, Chiikawa Momonga plush, Chiikawa Kurimanju plush, Pat and Mat Pat plush, Pat and Mat Mat plush, How to Train Your Dragon Toothless plush, How to Train Your Dragon Light Fury plush, How to Train Your Dragon Hiccup plush, How to Train Your Dragon Astrid plush, How to Train Your Dragon Night Fury plush, How to Train Your Dragon Stormfly plush, One Piece Luffy plush, One Piece Zoro plush, One Piece Nami plush, One Piece Usopp plush, One Piece Sanji plush, One Piece Chopper plush, One Piece Robin plush, One Piece Franky plush, One Piece Brook plush, One Piece Jinbe plush, One Piece Law plush, One Piece Ace plush, One Piece Shanks plush, One Piece Sabo plush, One Piece Hancock plush, One Piece Yamato plush, One Piece Bepo plush, Pororo Pororo plush, Pororo Eddy plush, Pororo Poby plush, Pororo Crong plush, Pororo Petty plush, Pororo Harry plush, Pororo Rody plush, Pororo Tongtong plush, Pororo Nyao plush, Mashimaro Mashimaro plush, Miffy Miffy plush, Miffy Melanie plush, Miffy Boris plush, Miffy Snuffy plush, Joking Bear plush, Pingu Pingu plush, Pingu Pinga plush, SpongeBob SpongeBob plush, SpongeBob Patrick plush, SpongeBob Squidward plush, SpongeBob Mr. Krabs plush, SpongeBob Sandy plush, SpongeBob Gary plush, Moomin Moomin plush, Moomin Snorkmaiden plush, Moomin Little My plush, Moomin Snufkin plush, Line Friends Brown plush, Line Friends Cony plush, Line Friends Sally plush, Kakao Friends Ryan plush, Kakao Friends Apeach plush, Kakao Friends Muzi plush, Kakao Friends Con plush, Kakao Friends Neo plush, Kakao Friends Frodo plush, Kakao Friends Tube plush, Kakao Friends Chunsik plush, Peppa Pig Peppa plush, Peppa Pig George plush, Super Mario Mario plush, Super Mario Luigi plush, Super Mario Princess Peach plush, Super Mario Toad plush, Super Mario Yoshi plush, Super Mario Bowser plush, Kirby Kirby plush, Kirby Meta Knight plush, Kirby Waddle Dee plush, Adventure Time Finn plush, Adventure Time Jake plush, Adventure Time BMO plush, Adventure Time Marceline plush, Peanuts Snoopy plush, Peanuts Charlie Brown plush, Peanuts Woodstock plush, Doraemon Doraemon plush, Doraemon Nobita plush, Doraemon Dorami plush, Anpanman Anpanman plush, Anpanman Baikinman plush, Anpanman Shokupanman plush, Thomas Thomas plush, Thomas Percy plush, Molang Molang plush, Molang Piu Piu plush, Esther Bunny Esther Bunny plush, Zanmang Loopy plush, Teletubbies Teletubbies plush, Teletubbies Po plush, Teletubbies Tinky Winky plush, Tayo Tayo plush, Tayo Rogi plush, Tayo Lani plush, Tayo Gani plush, Robocar Poli Poli plush, Robocar Poli Roy plush, Robocar Poli Amber plush, Robocar Poli Helly plush, Cocomong Cocomong plush, Catch Teenieping Heartsping plush, Catch Teenieping Teenieping plush, Catch Teenieping Azaping plush, Super Wings Jett plush, Super Wings plush, Minions Bob plush, Minions Kevin plush, Minions Stuart plush, Minions plush, Dragon Ball Goku plush, Dragon Ball Vegeta plush, Demon Slayer Tanjiro plush, Demon Slayer Nezuko plush, Demon Slayer Zenitsu plush, Demon Slayer Inosuke plush, Demon Slayer Rengoku plush, Naruto Naruto plush, Naruto Sasuke plush, Naruto Kakashi plush, Naruto Kurama plush, BT21 Tata plush, BT21 Chimmy plush, BT21 Cooky plush, BT21 Mang plush, BT21 Koya plush, BT21 plush,
ディズニー ミッキーマウス ぬいぐるみ, ディズニー ミニーマウス ぬいぐるみ, ディズニー ドナルドダック ぬいぐるみ, ディズニー デイジーダック ぬいぐるみ, ディズニー グーフィー ぬいぐるみ, ディズニー プルート ぬいぐるみ, ディズニー チップ ぬいぐるみ, ディズニー デール ぬいぐるみ, ディズニー エルサ ぬいぐるみ, ディズニー アナ ぬいぐるみ, ディズニー オラフ ぬいぐるみ, ディズニー ラプンツェル ぬいぐるみ, ディズニー モアナ ぬいぐるみ, ディズニー スティッチ ぬいぐるみ, ディズニー プーさん ぬいぐるみ, ディズニー くまのプーさん ぬいぐるみ, ディズニー ピグレット ぬいぐるみ, ディズニー ティガー ぬいぐるみ, ディズニー イーヨー ぬいぐるみ, ディズニー シンバ ぬいぐるみ, ディズニー ナラ ぬいぐるみ, ディズニー プンバァ ぬいぐるみ, ディズニー ティモン ぬいぐるみ, ディズニー バズ ぬいぐるみ, ディズニー ウッディ ぬいぐるみ, ディズニー ジェシー ぬいぐるみ, ディズニー レックス ぬいぐるみ, ディズニー フォーキー ぬいぐるみ, ディズニー ハム ぬいぐるみ, ディズニー エイリアン ぬいぐるみ, ディズニー マイク ぬいぐるみ, ディズニー サリー ぬいぐるみ, ディズニー ウォーリー ぬいぐるみ, ディズニー イヴ ぬいぐるみ, ディズニー ニモ ぬいぐるみ, ディズニー ドリー ぬいぐるみ, ディズニー レミー ぬいぐるみ, ディズニー ベイマックス ぬいぐるみ, ディズニー ラルフ ぬいぐるみ, ディズニー ヴァネロペ ぬいぐるみ, ディズニー ジュディ ぬいぐるみ, ディズニー ニック ぬいぐるみ, ディズニー アラジン ぬいぐるみ, ディズニー ジャスミン ぬいぐるみ, ディズニー ジーニー ぬいぐるみ, ディズニー シンデレラ ぬいぐるみ, ディズニー 白雪姫 ぬいぐるみ, ディズニー オーロラ ぬいぐるみ, ディズニー アリエル ぬいぐるみ, ディズニー フランダー ぬいぐるみ, ディズニー セバスチャン ぬいぐるみ, ディズニー ベル ぬいぐるみ, ディズニー 野獣 ぬいぐるみ, ディズニー ムーラン ぬいぐるみ, ディズニー ムーシュー ぬいぐるみ, ディズニー ティアナ ぬいぐるみ, ディズニー メリダ ぬいぐるみ, ディズニー ダンボ ぬいぐるみ, ディズニー バンビ ぬいぐるみ, ディズニー ピノキオ ぬいぐるみ, ディズニー アリス ぬいぐるみ, ディズニー チェシャ猫 ぬいぐるみ, ディズニー ミラベル ぬいぐるみ, ディズニー ブルーノ ぬいぐるみ, ディズニー ルイーサ ぬいぐるみ, ディズニー イサベラ ぬいぐるみ, ディズニー ルカ ぬいぐるみ, ディズニー アルベルト ぬいぐるみ, ディズニー アーシャ ぬいぐるみ, サンリオ ハローキティ ぬいぐるみ, サンリオ マイメロディ ぬいぐるみ, サンリオ クロミ ぬいぐるみ, サンリオ シナモロール ぬいぐるみ, サンリオ ポムポムプリン ぬいぐるみ, サンリオ ポチャッコ ぬいぐるみ, サンリオ けろけろけろっぴ ぬいぐるみ, サンリオ バッドばつ丸 ぬいぐるみ, サンリオ ハンギョドン ぬいぐるみ, サンリオ リトルツインスターズ ぬいぐるみ, サンリオ キキ ぬいぐるみ, サンリオ ララ ぬいぐるみ, サンリオ マロンクリーム ぬいぐるみ, サンリオ ウサハナ ぬいぐるみ, サンリオ ぐでたま ぬいぐるみ, サンリオ マイスウィートピアノ ぬいぐるみ, ジブリ トトロ ぬいぐるみ, ジブリ 中トトロ ぬいぐるみ, ジブリ 小トトロ ぬいぐるみ, ジブリ ネコバス ぬいぐるみ, ジブリ ジジ ぬいぐるみ, ジブリ 黒猫ジジ ぬいぐるみ, ジブリ カオナシ ぬいぐるみ, ジブリ ハク ぬいぐるみ, ジブリ 千尋 ぬいぐるみ, ジブリ 湯婆婆 ぬいぐるみ, ジブリ ポニョ ぬいぐるみ, ジブリ 宗介 ぬいぐるみ, ジブリ ハウル ぬいぐるみ, ジブリ ソフィー ぬいぐるみ, ジブリ カルシファー ぬいぐるみ, ジブリ サン ぬいぐるみ, ジブリ アシタカ ぬいぐるみ, ジブリ ヤックル ぬいぐるみ, ジブリ こだま ぬいぐるみ, ジブリ ナウシカ ぬいぐるみ, ジブリ テト ぬいぐるみ, ジブリ メイ ぬいぐるみ, ジブリ サツキ ぬいぐるみ, ジブリ アリエッティ ぬいぐるみ, ジブリ バロン ぬいぐるみ, ジブリ ムタ ぬいぐるみ, ポケモン ピカチュウ ぬいぐるみ, ポケモン イーブイ ぬいぐるみ, ポケモン ヒトカゲ ぬいぐるみ, ポケモン リザードン ぬいぐるみ, ポケモン ゼニガメ ぬいぐるみ, ポケモン カメックス ぬいぐるみ, ポケモン フシギダネ ぬいぐるみ, ポケモン カビゴン ぬいぐるみ, ポケモン ミュウ ぬいぐるみ, ポケモン ミュウツー ぬいぐるみ, ポケモン カイリュー ぬいぐるみ, ポケモン プリン ぬいぐるみ, ポケモン ピチュー ぬいぐるみ, ポケモン ルカリオ ぬいぐるみ, ポケモン ルギア ぬいぐるみ, ポケモン ホウオウ ぬいぐるみ, ポケモン ガーディ ぬいぐるみ, ポケモン ウインディ ぬいぐるみ, ポケモン コイキング ぬいぐるみ, ポケモン ギャラドス ぬいぐるみ, ポケモン エモンガ ぬいぐるみ, ポケモン コダック ぬいぐるみ, ポケモン ニンフィア ぬいぐるみ, ポケモン ブラッキー ぬいぐるみ, ポケモン グレイシア ぬいぐるみ, ポケモン リーフィア ぬいぐるみ, ポケモン ブースター ぬいぐるみ, ポケモン シャワーズ ぬいぐるみ, ポケモン サンダース ぬいぐるみ, ポケモン メタモン ぬいぐるみ, ポケモン ライチュウ ぬいぐるみ, クレヨンしんちゃん しんちゃん ぬいぐるみ, クレヨンしんちゃん シロ ぬいぐるみ, クレヨンしんちゃん 風間くん ぬいぐるみ, クレヨンしんちゃん ネネちゃん ぬいぐるみ, クレヨンしんちゃん マサオくん ぬいぐるみ, クレヨンしんちゃん ボーちゃん ぬいぐるみ, クレヨンしんちゃん ひまわり ぬいぐるみ, クレヨンしんちゃん みさえ ぬいぐるみ, クレヨンしんちゃん ひろし ぬいぐるみ, クレヨンしんちゃん アクション仮面 ぬいぐるみ, ぼのぼの ぼのぼの ぬいぐるみ, ぼのぼの アライグマ ぬいぐるみ, ぼのぼの しまりす ぬいぐるみ, ちいかわ ちいかわ ぬいぐるみ, ちいかわ ハチワレ ぬいぐるみ, ちいかわ うさぎ ぬいぐるみ, ちいかわ モモンガ ぬいぐるみ, ちいかわ 栗まんじゅう ぬいぐるみ, パットとマット パット ぬいぐるみ, パットとマット マット ぬいぐるみ, ヒックとドラゴン トゥース ぬいぐるみ, ヒックとドラゴン ライトフューリー ぬいぐるみ, ヒックとドラゴン ヒック ぬいぐるみ, ヒックとドラゴン アスティ ぬいぐるみ, ヒックとドラゴン ナイトフューリー ぬいぐるみ, ヒックとドラゴン ストームフライ ぬいぐるみ, ワンピース ルフィ ぬいぐるみ, ワンピース ゾロ ぬいぐるみ, ワンピース ナミ ぬいぐるみ, ワンピース ウソップ ぬいぐるみ, ワンピース サンジ ぬいぐるみ, ワンピース チョッパー ぬいぐるみ, ワンピース ロビン ぬいぐるみ, ワンピース フランキー ぬいぐるみ, ワンピース ブルック ぬいぐるみ, ワンピース ジンベエ ぬいぐるみ, ワンピース ロー ぬいぐるみ, ワンピース エース ぬいぐるみ, ワンピース シャンクス ぬいぐるみ, ワンピース サボ ぬいぐるみ, ワンピース ハンコック ぬいぐるみ, ワンピース ヤマト ぬいぐるみ, ワンピース ベポ ぬいぐるみ, ポロロ ポロロ ぬいぐるみ, ポロロ エディ ぬいぐるみ, ポロロ ポビ ぬいぐるみ, ポロロ クロン ぬいぐるみ, ポロロ ペティ ぬいぐるみ, ポロロ ハリー ぬいぐるみ, ポロロ ロディ ぬいぐるみ, マシマロ マシマロ ぬいぐるみ, ミッフィー ミッフィー ぬいぐるみ, ミッフィー メラニー ぬいぐるみ, ミッフィー ボリス ぬいぐるみ, ミッフィー スナッフィー ぬいぐるみ, ピングー ピングー ぬいぐるみ, ピングー ピンガ ぬいぐるみ, スポンジボブ スポンジボブ ぬいぐるみ, スポンジボブ パトリック ぬいぐるみ, スポンジボブ イカルド ぬいぐるみ, スポンジボブ カーニさん ぬいぐるみ, スポンジボブ サンディ ぬいぐるみ, スポンジボブ ゲイリー ぬいぐるみ, ムーミン ムーミン ぬいぐるみ, ムーミン スノークのおじょうさん ぬいぐるみ, ムーミン リトルミイ ぬいぐるみ, ムーミン スナフキン ぬいぐるみ, ラインフレンズ ブラウン ぬいぐるみ, ラインフレンズ コニー ぬいぐるみ, ラインフレンズ サリー ぬいぐるみ, カカオフレンズ ライアン ぬいぐるみ, カカオフレンズ アピーチ ぬいぐるみ, カカオフレンズ ムジ ぬいぐるみ, カカオフレンズ コン ぬいぐるみ, カカオフレンズ ネオ ぬいぐるみ, カカオフレンズ フロド ぬいぐるみ, カカオフレンズ チューブ ぬいぐるみ, カカオフレンズ チュンシク ぬいぐるみ, ペッパピッグ ペッパ ぬいぐるみ, ペッパピッグ ジョージ ぬいぐるみ, スーパーマリオ マリオ ぬいぐるみ, スーパーマリオ ルイージ ぬいぐるみ, スーパーマリオ ピーチ姫 ぬいぐるみ, スーパーマリオ キノピオ ぬいぐるみ, スーパーマリオ ヨッシー ぬいぐるみ, スーパーマリオ クッパ ぬいぐるみ, 星のカービィ カービィ ぬいぐるみ, 星のカービィ メタナイト ぬいぐるみ, 星のカービィ ワドルディ ぬいぐるみ, アドベンチャータイム フィン ぬいぐるみ, アドベンチャータイム ジェイク ぬいぐるみ, アドベンチャータイム ビーモ ぬいぐるみ, アドベンチャータイム マーセリン ぬいぐるみ, ピーナッツ スヌーピー ぬいぐるみ, ピーナッツ チャーリーブラウン ぬいぐるみ, ピーナッツ ウッドストック ぬいぐるみ, ドラえもん ドラえもん ぬいぐるみ, ドラえもん のび太 ぬいぐるみ, ドラえもん ドラミ ぬいぐるみ, アンパンマン アンパンマン ぬいぐるみ, アンパンマン ばいきんまん ぬいぐるみ, アンパンマン しょくぱんまん ぬいぐるみ, きかんしゃトーマス トーマス ぬいぐるみ, きかんしゃトーマス パーシー ぬいぐるみ, モラン モラン ぬいぐるみ, モラン ピウピウ ぬいぐるみ, エスターバニー エスターバニー ぬいぐるみ, テレタビーズ テレタビーズ ぬいぐるみ, テレタビーズ ポー ぬいぐるみ, テレタビーズ ティンキーウィンキー ぬいぐるみ, タヨ タヨ ぬいぐるみ, タヨ ロギ ぬいぐるみ, タヨ ラニ ぬいぐるみ, タヨ ガニ ぬいぐるみ, ロボカーポリー ポリー ぬいぐるみ, ロボカーポリー ロイ ぬいぐるみ, ロボカーポリー アンバー ぬいぐるみ, ロボカーポリー ヘリー ぬいぐるみ, ココモン ココモン ぬいぐるみ, キャッチティニピン ハーツピン ぬいぐるみ, キャッチティニピン ティニピン ぬいぐるみ, スーパーウィングス ジェット ぬいぐるみ, ミニオンズ ボブ ぬいぐるみ, ミニオンズ ケビン ぬいぐるみ, ミニオンズ スチュアート ぬいぐるみ, ミニオンズ ミニオン ぬいぐるみ, ドラゴンボール 悟空 ぬいぐるみ, ドラゴンボール ベジータ ぬいぐるみ, 鬼滅の刃 炭治郎 ぬいぐるみ, 鬼滅の刃 禰豆子 ぬいぐるみ, 鬼滅の刃 善逸 ぬいぐるみ, 鬼滅の刃 伊之助 ぬいぐるみ, 鬼滅の刃 煉獄 ぬいぐるみ, ナルト ナルト ぬいぐるみ, ナルト サスケ ぬいぐるみ, ナルト カカシ ぬいぐるみ, ナルト 九喇嘛 ぬいぐるみ, BT21 タタ ぬいぐるみ, BT21 チミー ぬいぐるみ, BT21 クッキー ぬいぐるみ, BT21 マン ぬいぐるみ, BT21 コヤ ぬいぐるみ, BT21 ぬいぐるみ,
미키마우스, 미니마우스, 도널드덕, 데이지덕, 구피, 플루토, 칩, 데일, 엘사, 안나, 올라프, 라푼젤, 모아나, 스티치, 푸우, 곰돌이푸, 피글렛, 티거, 이요르, 심바, 날라, 품바, 티몬, 버즈, 우디, 제시, 렉스, 포키, 햄, 에일리언, 마이크, 설리, 월E, 이브, 니모, 도리, 레미, 베이맥스, 주먹왕랄프, 바넬로피, 주디홉스, 닉와일드, 알라딘, 자스민, 지니, 신데렐라, 백설공주, 오로라, 아리엘, 플라운더, 세바스찬, 벨, 야수, 뮬란, 무슈, 티아나, 메리다, 덤보, 밤비, 피노키오, 앨리스, 체셔캣, 미라벨, 브루노, 루이자, 이사벨라, 루카, 알베르토, 아샤, 헬로키티, 마이멜로디, 쿠로미, 시나모롤, 폼폼푸린, 포차코, 케로케로케로피, 배드바츠마루, 한교동, 리틀트윈스타즈, 키키, 라라, 마론크림, 우사하나, 구데타마, 마이스위트피아노, 토토로, 중토토로, 소토토로, 고양이버스, 지지, 검은고양이지지, 가오나시, 하쿠, 치히로, 유바바, 포뇨, 소스케, 하울, 소피, 캘시퍼, 산, 아시타카, 야쿠루, 코다마, 나우시카, 테토, 메이, 사츠키, 아리에티, 바론, 무타, 피카츄, 이브이, 파이리, 리자몽, 꼬부기, 거북왕, 이상해씨, 잠만보, 뮤, 뮤츠, 망나뇽, 푸린, 피츄, 루카리오, 루기아, 칠색조, 가디, 윈디, 잉어킹, 갸라도스, 에몽가, 프시덕, 님피아, 블래키, 글레이시아, 리피아, 부스터, 샤워즈, 쥬피썬더, 메타몽, 라이츄, 짱구, 신짱, 흰둥이, 시로, 철수, 유리, 훈이, 맹구, 짱아, 봉미선, 신형만, 채성아, 수지, 액션가면, 보노보노, 너구리, 포로리, 시마리스, 아라이구마, 치이카와, 하치와레, 우사기, 모몽가, 쿠리만주, 먼작귀, 패트, 매트, 패트와매트, 투슬리스, 라이트퓨리, 히컵, 아스트리드, 나이트퓨리, 스톰플라이, 루피, 조로, 나미, 우솝, 상디, 쵸파, 로빈, 프랑키, 브룩, 징베, 로우, 에이스, 샹크스, 사보, 핸콕, 야마토, 베포, 뽀로로, 에디, 포비, 크롱, 패티, 해리, 로디, 통통이, 뉴티, 마시마로, 마시멜로, 미피, 미피토끼, 멜라니, 보리스, 스너피, 농담곰, 핑구, 핑가, 스폰지밥, 뚱이, 징징이, 집게사장, 다람이, 게리, 무민, 스노크메이든, 리틀미이, 스너프킨, 브라운, 코니, 샐리, 라이언, 어피치, 무지, 콘, 네오, 프로도, 튜브, 춘식이, 페파피그, 조지, 마리오, 루이지, 피치공주, 키노피오, 요시, 쿠파, 커비, 메타나이트, 와들디, 핀, 제이크, 비모, 마셀린, 스누피, 찰리브라운, 우드스탁, 도라에몽, 노비타, 도라미, 호빵맨, 세균맨, 식빵맨, 토마스, 퍼시, 몰랑, 몰랑이, 피우피우, 빵빵이, 에스더버니, 잔망루피, 텔레토비, 뽀, 보라돌이, 타요, 로기, 라니, 가니, 폴리, 로이, 앰버, 헬리, 코코몽, 하츄핑, 티니핑, 아자핑, 호기, 슈퍼윙스, 밥, 케빈, 스튜어트, 미니언즈, 손오공, 베지터, 탄지로, 네즈코, 젠이츠, 이노스케, 렌고쿠, 나루토, 사스케, 카카시, 쿠라마, 타타, 치미, 쿠키, 망, 코야, BT21
""" # @param {type:"string"}
TARGET_OBJECT = "doll" # @param {type:"string"}
LIMIT = 50000 # @param {type:"integer"}

# 키워드 파싱 (공백으로 구분하거나, 선호하는 경우 단일 구문으로 처리)
# CLI는 여러 인수를 리스트로 처리. 여기서는 하나의 문자열을 받아 필요시 분할하거나
# 단순히 리스트로 래핑.
keyword_list = [k.strip() for k in KEYWORDS.split(',') if k.strip()]
if not keyword_list:
    keyword_list = [KEYWORDS]

print(f"처리 중인 키워드: {keyword_list}")
print(f"대상 객체: {TARGET_OBJECT}")
print(f"제한: {LIMIT}")
print(f"저장 경로: {settings.output_dir}")

def export_results(job_id: str):
    if not os.path.isdir(LOCAL_OUTPUT_DIR) and not os.path.isdir("data"):
        print("[WARN] 로컬 출력 디렉토리와 data/가 모두 없습니다. 내보내기를 건너뜁니다.")
        return

    os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    archive_name = f"{job_id}_{timestamp}.tar.gz"
    archive_path = os.path.join("/content", archive_name)

    print(f"로컬 결과 압축 중... {archive_path}")
    with tarfile.open(archive_path, "w:gz") as tar:
        if os.path.isdir(LOCAL_OUTPUT_DIR):
            local_base = os.path.basename(LOCAL_OUTPUT_DIR.rstrip('/'))
            tar.add(LOCAL_OUTPUT_DIR, arcname=local_base)
        if os.path.isdir("data"):
            tar.add("data", arcname="data")

    drive_archive = os.path.join(DRIVE_OUTPUT_DIR, archive_name)
    shutil.copy2(archive_path, drive_archive)
    print(f"Drive로 압축본 복사 완료: {drive_archive}")

def run_pipeline():
    # Job ID 생성
    first_kw = keyword_list[0].replace(' ', '_')
    job_id = f"job_{TARGET_OBJECT}_{first_kw}"
    
    # 기존 작업 데이터 확인하여 실수로 덮어쓰기/재실행 방지
    # (파이프라인 로직 자체가 건너뛰기를 이상적으로 처리하지만 경고)
    job_path = os.path.join(settings.output_dir, job_id)
    if os.path.exists(job_path):
        print(f"\n[INFO] 작업 디렉토리 {job_path}가 이미 존재합니다. 파이프라인이 재개하거나 완료된 단계를 건너뛰려고 시도합니다.")
    
    # Job 생성
    job = Job(
        keywords=keyword_list,
        target_class=TARGET_OBJECT,
        limit=LIMIT,
        job_id=job_id
    )
    
    # 실행
    runner = PipelineRunner()
    try:
        runner.run(job)
        print("\n대성공! 파이프라인이 완료되었습니다.")
        return True, job_id
    except Exception as e:
        print(f"\n[ERROR] 파이프라인 실패: {e}")
        logging.exception("Pipeline Failure")
        return False, job_id

# 실행
if __name__ == "__main__":
    ok, job_id = run_pipeline()
    if ok:
        export_results(job_id)
